# Water scarcity assessment


## Setup

### Import libraries


In [ ]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

from functions.data_etl.file_io import read_gdf_from_csv
from functions.project_settings import (
    GLOBAL_WARMING_SCENARIOS,
    GLOBAL_WARMING_SCENARIOS_FUTURE,
    MOLLWEIDE_CRS,
    WGS84_CRS,
)
from functions.water_scarcity.analysis import (
    calculate_monthly_discharge_abstraction,
    calculate_water_scarcity,
    compute_efr,
    find_water_extraction_sites,
    get_global_warming_scenario_files,
    get_water_scarcity_summary,
    repeat_rows_for_months,
    summing_dc_water_use_per_extraction_site,
)
from functions.water_scarcity.pcrglobwb_processing import (
    process_abstraction_rasters,
    process_discharge_raster,
    reproject_rasters_to_crs,
)


## Setting input and output paths


In [2]:
# Input paths
INPUTS_DIR = Path("data/inputs/3_water_scarcity/")
WATER_USE_BY_LOCATION = Path("data/outputs/2_energy_and_water_use/water_use_dcs_pps_baseline.csv")

# Output paths
OUTPUT_DIR = Path("data/outputs/3_water_scarcity/")
FIGURE_DIR = Path("data/outputs/figures/")
PROCESSED_PCRGLOBWB_DIR = OUTPUT_DIR / "pcrglobwb_extracted_values/"
INTERMEDIATE_WATER_USE_OUTPUT_PATH = OUTPUT_DIR / "water_use_by_location_with_water_extraction_locations.csv"

## Processing PCRGLOB-WB 2.0 files


We use PCRGLOBWB discharge, ground and surface water abstraction rasters from [FutureStreams](https://github.com/JoyceBosmans/FutureStreams/tree/main) for various global warming scenarios (historical, 1.5C, 2.0C, 3.2C, 4.5C)


The rasters are first reprojected to WGS84 using the `gdal.Warp` function from the `rasterio` library and saved in the `outputs` folder.


### Reprojecting input raster files


In [ ]:
# Setting in- and output directories
reproj_input_dir = INPUTS_DIR / "pcrglobwb_processed"
reproj_output_dir = OUTPUT_DIR / "pcrglobwb_extracted_values/reprojected"

# Get all .tif files in the input directory
src_paths = list(Path(reproj_input_dir).glob("*.tif"))

# Reproject all .tif files to WGS84
reproject_rasters_to_crs(src_paths, reproj_output_dir, crs=WGS84_CRS)

The reprojected rasters are then processed and the data is stored in the `outputs` dictionary, divided by global warming targets.


In [ ]:
# Get all reprojected rasters
reprojected_raster_paths = list(Path(reproj_output_dir).glob("*.tif"))

# Sort rasters by type using dictionary comprehension
raster_types = {
    "discharge": [p for p in reprojected_raster_paths if p.name.startswith("q")],
    "surface": [p for p in reprojected_raster_paths if p.name.startswith("sAb")],
    "groundwater": [p for p in reprojected_raster_paths if p.name.startswith("gAb")],
}

# Create pairs of surface and groundwater abstraction rasters
abstraction_raster_pairs = [
    (s_path, g_path)
    for s_path in raster_types["surface"]
    for g_path in raster_types["groundwater"]
    if s_path.name.split("sAb")[1] == g_path.name.split("gAb")[1]
]

### Processing reprojected raster files


In [ ]:
output_directory_processed = OUTPUT_DIR / "pcrglobwb_extracted_values23"

# TODO: Instead of handling all rasters separately, flatten them into min, max, median, rasters
# per global warming scenario here and use these directly later on

# Process all the discharge rasters
for discharge_raster in tqdm(raster_types["discharge"]):
    print(f"Processing discharge raster: {discharge_raster}")
    process_discharge_raster(discharge_raster, output_directory_processed)

In [ ]:
# Process all the abstraction rasters
for surface_water_abstraction_raster, groundwater_abstraction_raster in tqdm(abstraction_raster_pairs):
    print(f"Processing abstraction rasters: {surface_water_abstraction_raster} and {groundwater_abstraction_raster}")
    process_abstraction_rasters(
        surface_water_abstraction_raster, groundwater_abstraction_raster, output_directory_processed
    )

## Water scarcity modeling


In [13]:
# Load in data center water use by location data
water_use_by_location = read_gdf_from_csv(WATER_USE_BY_LOCATION)

### Locating water extraction sites for data centers and power plants.

Data centers are assumed to withdraw water consumed from the cell within a 30 kilometer radius where the highest average discharge occurs throughout the year. For power plants, a 10 km buffer is used. However, 30 km is also used for solar and wind plants as they are often located further from water resources

This process is performed based on the future dataset as planned data centers locations are included. The identified extraction sites for the operational data centers is then merged for the historical scenario.


In [5]:
# Set buffer distances (in meters)
BUFFER_SIZES = {
    "data_center": 30_000,
    "Solar": 30_000,
    "Wind": 30_000,
    "default": 10_000,  # Default for other power plants
}

# Project to a metric crs (Mollweide)
water_use_by_location = water_use_by_location.to_crs(MOLLWEIDE_CRS)

# Create buffers around the points
water_use_by_location["geometry"] = water_use_by_location.apply(
    lambda x: x["geometry"].buffer(BUFFER_SIZES.get(x.type, BUFFER_SIZES["default"])), axis=1
)

# Reproject to WGS84
water_use_by_location = water_use_by_location.to_crs(WGS84_CRS)

All 5 climate models produced the same place as the highest flow cell within the buffer. Therefore, we run it only using 1 raster file (and hence climate model) to base the water extraction locations on.


In [6]:
# Get first historical discharge raster
discharge_file_hist = next(Path(PROCESSED_PCRGLOBWB_DIR, "hist").glob("q*"))

# Assign water extraction points to each data center and power plant
water_use_by_location = find_water_extraction_sites(water_use_by_location, discharge_file_hist)

Processing locations: 100%|██████████| 38835/38835 [20:08<00:00, 32.12it/s] 


As the the assignment of water extraction sites takes a long time (20 min), we will save the results to a csv and load it in the future.


In [ ]:
# Export to a csv file
water_use_by_location.to_csv(INTERMEDIATE_WATER_USE_OUTPUT_PATH, index=False)

### Calculating cumulative water use at water extraction sites


In [3]:
# Import the water use by location data
water_use_by_location = read_gdf_from_csv(INTERMEDIATE_WATER_USE_OUTPUT_PATH)

In [4]:
# Finding the cumulative data-center-driven water use at extraction sites
water_use_by_location = summing_dc_water_use_per_extraction_site(water_use_by_location)

### Assigning monthly discharge and abstraction values to extraction sites


In [5]:
# Repeat each row in for all 12 months so that we can calculate the water scarcity for each month
water_use_by_location = repeat_rows_for_months(water_use_by_location)

In [6]:
# Process all scenarios
discharge_abstraction_by_location = {}
for scenario in GLOBAL_WARMING_SCENARIOS:
    # Get folder path and files
    discharge_files, abstraction_files = get_global_warming_scenario_files(PROCESSED_PCRGLOBWB_DIR / scenario)

    # Calculate discharge and abstraction
    result = calculate_monthly_discharge_abstraction(
        water_use_by_location,
        discharge_files,
        abstraction_files,
        "lat_water_extraction",
        "lon_water_extraction",
    )

    # Store results
    discharge_abstraction_by_location[scenario] = result

Processing files: 100%|██████████| 7/7 [00:15<00:00,  2.24s/it]


### Environmental flow requirements


EFR ratios are assigned based on the VMF method. They are calculated in m3/month based on historical data, and the absolute flow is applied to future scenarios. As a sensitivity analysis, a consistent EFR 60% threshold was also calculated based on historical data and the absolute EFR was applied to future scenarios.


In [7]:
# Calculate EFRs based on historical data
discharge_abstraction_by_location["hist"] = compute_efr(discharge_abstraction_by_location["hist"])

# Assign the historical EFR to the future scenarios
for scenario in GLOBAL_WARMING_SCENARIOS_FUTURE:
    discharge_abstraction_by_location[scenario]["EFR_m3_month"] = discharge_abstraction_by_location["hist"][
        "EFR_m3_month"
    ]
    # Assign the 0.6 EFR to the future scenarios as a sensitivity analysis
    discharge_abstraction_by_location[scenario]["EFR_m3_month_0p6"] = discharge_abstraction_by_location["hist"][
        "EFR_m3_month_0p6"
    ]

### Water scarcity modeling

Data on discharge and abstraction comes from the PCRGLOB-WB 2.0 model

Water scarcity is calculated by

Water Scarcity Index = $\frac{\text{Abstraction}}{(\text{Discharge} + \text{Abstraction}) - \text{EFR}}$

An extraction site is considered water-scarce if the index surpasses 1 for at least one month of the year.

#### Vulnerability

A data center is considered vulnerable to water shortages if the abstraction exceeds total water availability, irregardless of EFR. This is calculated as

Vulnerability Index = $\frac{\text{Abstraction}}{\text{Discharge} + \text{Abstraction}}$

A data center is considered vulnerable if the index surpasses 1 for at least one month of the year. The months of vulnerability are compared between future and historical scenarios.

#### Contribution

Water scarcity accounting for data center water use is calculated by:

$\frac{\text{Abstraction} + \text{Data-center-driven water use}}{(\text{Discharge} + \text{Abstraction}) - \text{EFR}}$

Discharge and abstraction data is by monthly mean, so we also calculate average water use per month by dividing by 12.


In [16]:
# Calculate water scarcity for  scenarios
water_scarcity_results = {}
for scenario in GLOBAL_WARMING_SCENARIOS:
    water_scarcity_results[scenario] = calculate_water_scarcity(
        df=discharge_abstraction_by_location[scenario],
        abstraction_column="abstraction_m3_median",
        discharge_column="discharge_m3_median",
        efr_column="EFR_m3_month",
        efr_sensitivity_analysis=True,
    )

In [17]:
# Columns to keep in the summary
columns_for_summary = [
    "name",
    "latitude",
    "longitude",
    "tcp_mw",
    "operational",
    "power_grid_zone",
    "dc_cumulative_monthly_water_use_m3",
    "shared_extraction_site",
    "power_scenario",
    "cooling_tech_scenario",
    "technology_performance_level",
    "type",
    "lon_water_extraction",
    "lat_water_extraction",
    "geometry",
]

In [18]:
# Summarize the water scarcity results
for scenario in GLOBAL_WARMING_SCENARIOS:
    water_scarcity_results[scenario] = get_water_scarcity_summary(
        water_scarcity_results[scenario], columns_for_summary, efr_sensitivity_analysis=True
    )

In [29]:
# Calculate the number of months of increase in months_WSI between the given scenario and historical scenario
for scenario in ["1_5C", "2_0C", "3_2C"]:
    water_scarcity_results[scenario]["months_WSI_increase"] = (
        water_scarcity_results[scenario]["months_WSI"] - water_scarcity_results["hist"]["months_WSI"]
    )
    water_scarcity_results[scenario]["months_WSI_0p6_increase"] = (
        water_scarcity_results[scenario]["months_WSI_0p6"] - water_scarcity_results["hist"]["months_WSI_0p6"]
    )

# Calculate the increase in months of vulnerability between the given scenario and historical scenario
for scenario in ["1_5C", "2_0C", "3_2C"]:
    water_scarcity_results[scenario]["months_vulnerability_increase"] = (
        water_scarcity_results[scenario]["months_vulnerability_dc"]
        - water_scarcity_results["hist"]["months_vulnerability_dc"]
    )
    water_scarcity_results[scenario]["months_vulnerability_0p6_increase"] = (
        water_scarcity_results[scenario]["months_vulnerability_dc_0p6"]
        - water_scarcity_results["hist"]["months_vulnerability_dc_0p6"]
    )

In [31]:
# Add a column to indicate the scenario
for scenario in GLOBAL_WARMING_SCENARIOS:
    water_scarcity_results[scenario]["scenario"] = scenario

In [ ]:
# Concatenate the results
water_scarcity_results_all = pd.concat([water_scarcity_results[scenario] for scenario in GLOBAL_WARMING_SCENARIOS])

In [36]:
# Export the results
water_scarcity_results_all.to_csv(OUTPUT_DIR / "water_scarcity_summary_all.csv", index=False)

In [25]:
# Export the water scarcity summary to a csv file
for scenario in GLOBAL_WARMING_SCENARIOS:
    water_scarcity_results[scenario].to_csv(f"{OUTPUT_DIR}/water_scarcity_summary_{scenario}.csv", index=False)

Without accounting for clusters, 150 dcs contribute to water scarcity in hist, 218 in 4.5, and 16600 power plants.
After accounting for clusters, 432 dcs contribute in hist, 573 in 4.5. 16665 for power plants.
For data centers, their cumulative impact in an area matters a lot.
